In [48]:
from datetime import datetime
import json
from os import environ
import requests

from dotenv import load_dotenv
import polars as pl

In [2]:
to_keep = [
    'abstract',
    'print_section',
    'print_page',
    'source',
#     'headline',
    'keywords',
    'pub_date',
    'document_type',
    'news_desk',
    'section_name',
    'subsection_name',
    'type_of_material',
    'word_count',
]

In [3]:
load_dotenv()

True

In [4]:
def query_nyt(year: int, month: int):   
    if len(str(year)) != 4:
           raise ValueError
    
    if month not in range(0, 13):
        raise ValueError
        
    url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={environ['NYT_KEY']}"
    
    return requests.get(url).json()

In [52]:
def clean_data(articles):
    data = []
    
    for k in to_keep:
        if k == 'keywords':
            data.append([[k['value'] for k in a.get(k, '')] for a in articles])
        else:
            data.append([a.get(k, '') for a in articles])
        
    return data

In [53]:
articles = query_nyt(2022, 1)['response']['docs']

In [54]:
clean_articles = clean_data(articles)

In [55]:
df = pl.DataFrame(clean_articles, columns=to_keep)

In [57]:
df

abstract,print_section,print_page,source,keywords,pub_date,document_type,news_desk,section_name,subsection_name,type_of_material,word_count
str,str,str,str,list[str],str,str,str,str,str,str,i64
"""The Crimson Ti...","""B""","""6""","""The New York T...","[""Football (College)"", ""University of Alabama"", ... ""College Football Playoff National Championship""]","""2022-01-01T00:...","""article""","""Sports""","""Sports""","""College Footba...","""News""",1245
"""The television...","""""","""""","""The New York T...","[""White, Betty"", ""People (Magazine)"", ... ""Television""]","""2022-01-01T00:...","""article""","""Business""","""Business Day""","""Media""","""News""",613
"""“The world loo...","""""","""""","""The New York T...","[""White, Betty"", ""Social Media"", ... ""Celebrities""]","""2022-01-01T00:...","""article""","""Express""","""Arts""","""Television""","""News""",443
"""Peter Wentz es...","""""","""""","""The New York T...","[""Crossword Puzzles"", ""Wentz, Peter (Crossword Constructor)""]","""2022-01-01T03:...","""article""","""Games""","""Crosswords & G...","""""","""News""",641
"""The next chall...","""""","""""","""The New York T...","[""Football (College)"", ""College Football Playoff National Championship"", ... ""University of Michigan""]","""2022-01-01T04:...","""article""","""Sports""","""Sports""","""College Footba...","""News""",1015
"""Streaking has ...","""D""","""6""","""The New York T...","[""Running"", ""Ultramarathon Running"", ... ""audio-neutral-immersive""]","""2022-01-01T05:...","""article""","""Sports""","""Sports""","""""","""News""",1192
"""And Georgia is...","""A""","""23""","""The New York T...","[""Football (College)"", ""College Football Playoff National Championship"", ... ""University of Georgia""]","""2022-01-01T05:...","""article""","""Sports""","""Sports""","""College Footba...","""News""",842
"""Eric Adams, th...","""A""","""1""","""The New York T...","[""New York City"", ""Adams, Eric L"", ... ""Content Type: Personal Profile""]","""2022-01-01T05:...","""article""","""Metro""","""New York""","""""","""News""",1777
"""Quotation of t...","""A""","""3""","""The New York T...",[],"""2022-01-01T05:...","""article""","""Summary""","""Today’s Paper""","""""","""Quote""",28


In [58]:
df.filter(df['source'] != "The New York Times")

abstract,print_section,print_page,source,keywords,pub_date,document_type,news_desk,section_name,subsection_name,type_of_material,word_count
str,str,str,str,list[str],str,str,str,str,str,str,i64


In [61]:
pl.get_dummies(df, columns=['keywords'])

ValueError: this operation is not implemented/valid for this dtype: List(Utf8)